In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel("/home/vishalthadari/Documents/Retail.xlsx")
df.head()
df1 = df

In [4]:
df1.Country.nunique()

38

In [5]:
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [6]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()

customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [7]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [8]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  23494
StockCode :  4065
Description :  4202
Quantity :  715
InvoiceDate :  21220
UnitPrice :  1574
CustomerID :  3950
Country :  1


In [9]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [10]:
df1['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [11]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [13]:
import datetime as dt
NOW = dt.datetime(2011,12,10)

df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [14]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [15]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,2,0.00
12747.0,2,103,4196.01
12748.0,0,4642,29072.10
12749.0,3,231,3868.20
12820.0,3,59,942.34


In [27]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles1 = quantiles.to_dict()
print(quantiles1)

{'recency': {0.25: 16.0, 0.5: 50.0, 0.75: 143.0}, 'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 101.0}, 'monetary_value': {0.25: 282.255, 0.5: 627.0600000000001, 0.75: 1521.7825}, 'r_quartile': {0.25: 1.0, 0.5: 2.0, 0.75: 3.0}, 'f_quartile': {0.25: 2.0, 0.5: 3.0, 0.75: 4.0}, 'm_quartile': {0.25: 1.25, 0.5: 2.5, 0.75: 3.75}}


In [28]:
segmented_rfm = rfmTable
print(segmented_rfm)

            recency  frequency  monetary_value  r_quartile  f_quartile  \
CustomerID                                                               
12346.0         325          2    0.000000e+00           4           4   
12747.0           2        103    4.196010e+03           1           1   
12748.0           0       4642    2.907210e+04           1           1   
12749.0           3        231    3.868200e+03           1           1   
12820.0           3         59    9.423400e+02           1           2   
12821.0         214          6    9.272000e+01           4           4   
12822.0          70         47    9.189800e+02           3           2   
12823.0          74          5    1.759500e+03           3           4   
12824.0          59         25    3.971200e+02           3           3   
12826.0           2         94    1.468120e+03           1           2   
12827.0           5         25    4.301500e+02           1           3   
12828.0           2         56    1.01

In [24]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [25]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

segmented_rfm.head()

TypeError: RScore() takes 1 positional argument but 3 were given

In [22]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str)+segmented_rfm.f_quartile.map(str)+segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,2,0.00,4,4,4,444
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4642,29072.10,1,1,1,111
12749.0,3,231,3868.20,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [23]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,433,256438.49,1,1,1,111
17450.0,8,351,187482.17,1,1,1,111
17511.0,2,1076,88125.38,1,1,1,111
16684.0,4,281,65892.08,1,1,1,111
13694.0,3,585,62653.10,1,1,1,111
15311.0,0,2491,59419.34,1,1,1,111
13089.0,2,1857,57385.88,1,1,1,111
14096.0,4,5128,57120.91,1,1,1,111
15061.0,3,410,54228.74,1,1,1,111
